# 1-4-FACSexpression-R
Jakke Neiro$^1$
1. Aboobaker laboratory, Department of Zoology, University of Oxford

## Contents of notebook
* 1. Introduction
* 2. Sleuth and normalisation factors
    * 2.1 Sleuth
    * 2.2 Transcripts to genes

## Files:
* Input: salmon_bootstrap directories
* Output: sleuth_df.csv, sleuth_gene.csv

# 1. Introduction

The Salmon pseudoalignment output was used to obtain TPM values for transcripts and genes. The TPM values were used in 1-3-FACSexpression 4. Proportional TPM values. 

# 2. Sleuth and normalisation factors

## 2.1 Sleuth

Sleuth and associated packages were uploaded:

In [5]:
library(wasabi)
library(annotables)
library(tidyverse)
library(sleuth)

Paths to Salmon directories were specified: 

In [7]:
sf_dirs <- file.path("/hydra/FACS/salmon_bootstrap", 
        c("rna_pearson_x1_na_na_SRR2009674_1.fastq.trimmed.fastq.gz",
          "rna_pearson_x1_na_na_SRR2009675_1.fastq.trimmed.fastq.gz",
          "rna_pearson_x2_na_na_SRR2009676_1.fastq.trimmed.fastq.gz",
          "rna_pearson_x2_na_na_SRR2009677_1.fastq.trimmed.fastq.gz",
          "rna_reddien_x1_na_000h_SRR1302023_1.fastq.trimmed.fastq.gz",
          "rna_reddien_x1_na_000h_SRR1302024_1.fastq.trimmed.fastq.gz",
          "rna_reddien_x2_na_000h_SRR1302025_1.fastq.trimmed.fastq.gz",
          "rna_reddien_x2_na_000h_SRR1302026_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_x1_na_na_SRR2407874_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_x1_na_na_SRR2407875_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_x1_na_na_SRR2407876_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_x1_na_na_SRR2407877_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_xins_na_na_SRR2407878_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_xins_na_na_SRR2407879_1.fastq.trimmed.fastq.gz",
          "rna_sanchez_xins_na_na_SRR2407880_1.fastq.trimmed.fastq.gz", 
          "rna_sanchez_xins_na_na_SRR2407881_1.fastq.trimmed.fastq.gz",
          "oenalx1",
          "oenalx2",
          "oenalxins"))
sf_dirs

[1] "../FACS/salmon_bootstrap/rna_pearson_x1_na_na_SRR2009674_1.fastq.trimmed.fastq.gz"  
 [2] "../FACS/salmon_bootstrap/rna_pearson_x1_na_na_SRR2009675_1.fastq.trimmed.fastq.gz"  
 [3] "../FACS/salmon_bootstrap/rna_pearson_x2_na_na_SRR2009676_1.fastq.trimmed.fastq.gz"  
 [4] "../FACS/salmon_bootstrap/rna_pearson_x2_na_na_SRR2009677_1.fastq.trimmed.fastq.gz"  
 [5] "../FACS/salmon_bootstrap/rna_reddien_x1_na_000h_SRR1302023_1.fastq.trimmed.fastq.gz"
 [6] "../FACS/salmon_bootstrap/rna_reddien_x1_na_000h_SRR1302024_1.fastq.trimmed.fastq.gz"
 [7] "../FACS/salmon_bootstrap/rna_reddien_x2_na_000h_SRR1302025_1.fastq.trimmed.fastq.gz"
 [8] "../FACS/salmon_bootstrap/rna_reddien_x2_na_000h_SRR1302026_1.fastq.trimmed.fastq.gz"
 [9] "../FACS/salmon_bootstrap/rna_sanchez_x1_na_na_SRR2407874_1.fastq.trimmed.fastq.gz"  
[10] "../FACS/salmon_bootstrap/rna_sanchez_x1_na_na_SRR2407875_1.fastq.trimmed.fastq.gz"  
[11] "../FACS/salmon_bootstrap/rna_sanchez_x1_na_na_SRR2407876_1.fastq.trimmed.fastq.gz"  
[12] "../FACS/salmon_bootstrap/rna_sanchez_x1_na_na_SRR2407877_1.fastq.trimmed.fastq.gz"  
[13] "../FACS/salmon_bootstrap/rna_sanchez_xins_na_na_SRR2407878_1.fastq.trimmed.fastq.gz"
[14] "../FACS/salmon_bootstrap/rna_sanchez_xins_na_na_SRR2407879_1.fastq.trimmed.fastq.gz"
[15] "../FACS/salmon_bootstrap/rna_sanchez_xins_na_na_SRR2407880_1.fastq.trimmed.fastq.gz"
[16] "../FACS/salmon_bootstrap/rna_sanchez_xins_na_na_SRR2407881_1.fastq.trimmed.fastq.gz"
[17] "../FACS/salmon_bootstrap/oenalx1"                                                   
[18] "../FACS/salmon_bootstrap/oenalx2"                                                   
[19] "../FACS/salmon_bootstrap/oenalxins"

In [13]:
prepare_fish_for_sleuth(sf_dirs)

In [9]:
length(sf_dirs)

[1] 19

In [10]:
sf_dirs_samples = c()
for (i in 1:16){
    sf_dirs_samples = c(sf_dirs_samples, strsplit(sf_dirs[i], "_")[[1]][7])
}
sf_dirs_samples = c(sf_dirs_samples, "oenalx1", "oenalx2", "oenalxins")
sf_dirs_samples

[1] "SRR2009674" "SRR2009675" "SRR2009676" "SRR2009677" "SRR1302023"
 [6] "SRR1302024" "SRR1302025" "SRR1302026" "SRR2407874" "SRR2407875"
[11] "SRR2407876" "SRR2407877" "SRR2407878" "SRR2407879" "SRR2407880"
[16] "SRR2407881" "oenalx1"    "oenalx2"    "oenalxins"

In [11]:
sampletype <- factor(c(c("x1", "x1", "x2", "x2", "x1", "x1", "x2", "x2"), rep("x1", 4), rep("xins", 4), c("x1", "x2", "xins")))
sampletype

[1] x1   x1   x2   x2   x1   x1   x2   x2   x1   x1   x1   x1   xins xins xins
[16] xins x1   x2   xins
Levels: x1 x2 xins

In [12]:
summarydata <- data.frame(sampletype, row.names = sf_dirs_samples)
all(sf_dirs_samples == rownames(summarydata))

[1] TRUE

In [15]:
names(sf_dirs) = sf_dirs_samples
sf_dirs

SRR2009674 
"../FACS/salmon_bootstrap/rna_pearson_x1_na_na_SRR2009674_1.fastq.trimmed.fastq.gz" 
                                                                         SRR2009675 
"../FACS/salmon_bootstrap/rna_pearson_x1_na_na_SRR2009675_1.fastq.trimmed.fastq.gz" 
                                                                         SRR2009676 
"../FACS/salmon_bootstrap/rna_pearson_x2_na_na_SRR2009676_1.fastq.trimmed.fastq.gz" 
                                                                         SRR2009677 
"../FACS/salmon_bootstrap/rna_pearson_x2_na_na_SRR2009677_1.fastq.trimmed.fastq.gz"

In [16]:
summarydata$sample <- rownames(summarydata)

In [17]:
summarydata$path <- sf_dirs

In [19]:
design <- ~ sampletype

Sleuth was used to process samples: 

In [ ]:
so <- sleuth_prep(summarydata, full_model = design, read_bootstrap_tpm = TRUE, extra_bootstrap_summary = TRUE)

In [ ]:
so2 = sleuth_fit(so)

TPM values were saved: 

In [ ]:
sleuth_matrix = sleuth_to_matrix(so2, "obs_norm", "tpm")
sleuth_df = data.frame(sleuth_matrix)
write.csv(sleuth_df, "../FACS/sleuth_df.csv")

## 2.2 Transcripts to genes

In [1]:
sleuth_df = read.csv("/hydra/FACS/sleuth_df.csv")

The table contains transcript IDs as rows and sample names as columns:

In [2]:
head(sleuth_df)

,X,oenalx1,oenalx2,oenalxins,SRR1302023,SRR1302024,SRR1302025,SRR1302026,SRR2009674,SRR2009675,SRR2009676,SRR2009677,SRR2407874,SRR2407875,SRR2407876,SRR2407877,SRR2407878,SRR2407879,SRR2407880,SRR2407881
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,MSTRG.1.2,8.2809785,17.3345828,6.2341117,1.987248,0.4025658,1.4679588,0.4341099,2.369642,1.403415,5.336480,3.266755,0.000000,1.035979,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.000000
2,MSTRG.1.3,0.0000000,0.0000000,0.4596998,0.000000,0.0000000,0.0000000,0.1097060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000000,0.0000000,0.000000,0.000000,0.2709511,0.000000
3,MSTRG.10.1,0.6112433,1.0445691,0.0000000,0.910830,2.5560896,0.7399325,1.3662308,1.380931,1.184396,1.413898,1.717768,1.266333,1.600544,0.8156264,0.5938287,1.382147,0.000000,0.5428738,0.000000
4,MSTRG.100.2,0.0000000,0.0000000,0.0000000,0.000000,5.3850583,0.0000000,0.0000000,0.000000,0.000000,4.540264,10.043158,0.000000,0.000000,13.3110117,9.0888437,0.000000,0.000000,13.7831643,0.000000
5,MSTRG.1000.1,0.2619087,0.4005167,0.0000000,0.000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000000,0.0000000,0.000000,0.000000,0.0000000,0.000000
6,MSTRG.10000.2,0.0000000,2.2529292,29.9933825,4.311170,1.1437629,10.6575430,17.0171720,5.842181,6.223460,14.907304,17.279939,4.237024,3.111474,0.0000000,0.0000000,12.600497,6.236188,3.1978721,5.097174


In [121]:
nrow(sleuth_df)

[1] 90773

In [22]:
gffcmp = read.table("../sexual_genome_annotation_files/ncrna_Neiro/gffcmp.stringtie_merged.gtf.tmap", head=TRUE)

In [24]:
head(gffcmp)

,ref_gene_id,ref_id,class_code,qry_gene_id,qry_id,num_exons,FPKM,TPM,cov,len,major_iso_id,ref_match_len
,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<fct>,<fct>
1,SMESG000026673.1,SMEST026673001.1,=,MSTRG.1,SMEST026673001.1,2,0,0,0,1072,SMEST026673001.1,553
2,SMESG000026673.1,SMEST026673001.1,m,MSTRG.1,MSTRG.1.2,1,0,0,0,1042,SMEST026673001.1,553
3,SMESG000026673.1,SMEST026673001.1,o,MSTRG.1,MSTRG.1.3,2,0,0,0,858,SMEST026673001.1,553
4,-,-,u,MSTRG.2,MSTRG.2.1,2,0,0,0,1569,MSTRG.2.1,-
5,-,-,u,MSTRG.2,MSTRG.2.2,3,0,0,0,1474,MSTRG.2.1,-
6,-,-,u,MSTRG.3,MSTRG.3.1,2,0,0,0,320,MSTRG.3.1,-


The transcripts were assigned to genes: 

In [109]:
sleuth_gene = sleuth_df

In [ ]:
for (i in 1:nrow(sleuth_gene)){
    subset_gene = subset(gffcmp ,gffcmp$qry_id == toString(sleuth_gene$X[i]))
    sleuth_gene[i] = toString(subset_gene$qry_gene_id[1])
}
write.csv(sleuth_gene, "../FACS/sleuth_gene.csv")

In [3]:
sleuth_gene = read.csv("/hydra/FACS/sleuth_gene.csv")

# FINNISHED